In [1]:
import json
import os.path
import pandas as pd
import requests
import wbgapi as wb
import functions as func

pd.set_option('future.no_silent_downcasting', True)

# country-codes: https://www.iban.com/country-codes
countries = ('AFG', 'ALB', 'DZA', 'ASM', 'AGO', 'ATG', 'ARG', 'ARM', 'ABW',
       'AUS', 'AUT', 'AZE', 'BHS', 'BHR', 'BGD', 'BRB', 'BLR', 'BEL',
       'BLZ', 'BEN', 'BMU', 'BTN', 'BOL', 'BIH', 'BWA', 'BRA', 'VGB',
       'BRN', 'BGR', 'BFA', 'BDI', 'KHM', 'CMR', 'CAN', 'CPV', 'CYM',
       'CAF', 'TCD', 'CHL', 'CHN', 'COL', 'COM', 'COG', 'COK', 'CRI',
       'CIV', 'HRV', 'CUB', 'CYP', 'CZE', 'COD', 'DNK', 'DJI', 'DMA',
       'DOM', 'ECU', 'EGY', 'SLV', 'GNQ', 'ERI', 'EST', 'SWZ', 'ETH',
       'FRO', 'FLK', 'FJI', 'FIN', 'FRA', 'GUF', 'PYF', 'GAB', 'GMB',
       'GEO', 'DEU', 'GHA', 'GIB', 'GRC', 'GRL', 'GRD', 'GLP', 'GUM',
       'GTM', 'GIN', 'GNB', 'GUY', 'HTI', 'HND', 'HKG', 'HUN', 'ISL',
       'IND', 'IDN', 'IRN', 'IRQ', 'IRL', 'ISR', 'ITA', 'JAM', 'JPN',
       'JOR', 'KAZ', 'KEN', 'KIR', 'KWT', 'KGZ', 'LAO', 'LVA', 'LBN',
       'LSO', 'LBR', 'LBY', 'LTU', 'LUX', 'MAC', 'MDG', 'MWI', 'MYS',
       'MDV', 'MLI', 'MLT', 'MTQ', 'MRT', 'MUS', 'MEX', 'MDA', 'MNG',
       'MNE', 'MSR', 'MAR', 'MOZ', 'MMR', 'NAM', 'NRU', 'NPL', 'NLD',
       'NCL', 'NZL', 'NIC', 'NER', 'NGA', 'NIU', 'PRK', 'MKD', 'NOR',
       'OMN', 'PAK', 'PSE', 'PAN', 'PNG', 'PRY', 'PER', 'PHL', 'POL',
       'PRT', 'PRI', 'QAT', 'REU', 'ROU', 'RUS', 'RWA', 'SHN', 'KNA',
       'LCA', 'SPM', 'VCT', 'WSM', 'STP', 'SAU', 'SEN', 'SRB', 'SYC',
       'SLE', 'SGP', 'SVK', 'SVN', 'SLB', 'SOM', 'ZAF', 'KOR', 'SSD',
       'ESP', 'LKA', 'SDN', 'SUR', 'SWE', 'CHE', 'SYR', 'TWN', 'TJK',
       'TZA', 'THA', 'TLS', 'TGO', 'TON', 'TTO', 'TUN', 'TUR', 'TKM',
       'TCA', 'UGA', 'UKR', 'ARE', 'GBR', 'USA', 'VIR', 'URY', 'UZB',
       'VUT', 'VEN', 'VNM', 'ESH', 'YEM', 'ZMB', 'ZWE')

url1 = 'data/ren_ren_65-22/03_modern-renewable-prod.csv'
df_energy_prod = pd.read_csv(url1)

In [2]:
# Dataframe Collection
df_names = ['df_energy_prod', 'df_energy_prod_world', 'df_energy_prod_regions', 'df_countries', 'df_energy', 'df_ember', 'df_wb']

# Create empty DataFrames if they don't exist
for name in df_names:
    if name not in globals():
        globals()[name] = pd.DataFrame()

## API definitions

In [4]:
# FastAPI by Ember
# https://api.ember-energy.org/v1/docs
base_url = 'https://api.ember-energy.org'
my_api_key = 'ddbb448b-f721-4fc7-9b52-68ad159bf482'

# dictionary with endpoints that may be necessary for the project.
EMBER_ENDPOINTS = {
    'generation_per_year': 'electricity-generation/yearly',
    'generation_per_month': 'electricity-generation/monthly',
    'demand_per_year': 'electricity-demand/yearly',
    'demand_per_month': 'electricity-demand/monthly'
}

In [5]:
# WBGAPI - World Bank Data
# !pip install wbgapi
# https://pypi.org/project/wbgapi/

# dictionary with extracted indicators that may be necessary for the project.
GDP_INDICATORS = {
    # Constant currency value indicators
    'gdp_constant': 'NY.GDP.MKTP.KD',
    'gdp_per_capita_constant': 'NY.GDP.PCAP.KD',
    'gdp_ppp_constant': 'NY.GDP.MKTP.PP.KD',
    'gdp_per_capita_PPP_constant': 'NY.GDP.PCAP.PP.KD',

    # Growth
    'gdp_growth_annual_percent': 'NY.GDP.MKTP.KD.ZG',

    # Current currency value indicators
    'gdp_current': 'NY.GDP.MKTP.CD',
    'gdp_per_capita_current': 'NY.GDP.PCAP.CD',
    'gdp_ppp_current': 'NY.GDP.MKTP.PP.CD',
    'gdp_per_capita_ppp_current': 'NY.GDP.PCAP.PP.CD',
}

## Clean Dataset

In [7]:
df_energy_prod = df_energy_prod.rename(columns={'Entity': 'country', 'Code': 'iso_code', 'Year': 'year', 'Electricity from wind (TWh)': 'wind', 'Electricity from hydro (TWh)': 'hydro', 'Electricity from solar (TWh)': 'solar', 'Other renewables including bioenergy (TWh)': 'other_inc_bio'})
# remove kosovo, udssr, regions and world & save regions and world in separate dfs
df_energy_prod = df_energy_prod.loc[df_energy_prod['iso_code'] != 'OWID_KOS']
df_energy_prod = df_energy_prod.loc[df_energy_prod['iso_code'] != 'OWID_USS']
df_energy_prod_world = df_energy_prod.loc[df_energy_prod['iso_code'] == 'OWID_WRL']
df_energy_prod = df_energy_prod.loc[df_energy_prod['iso_code'] != 'OWID_WRL']
df_energy_prod_regions = df_energy_prod[df_energy_prod['iso_code'].isna()]
df_energy_prod = df_energy_prod.loc[df_energy_prod['iso_code'].notna()]
df_energy_prod = df_energy_prod.loc[df_energy_prod['year'] != 2022]

In [8]:
# create countries dataframe
if os.path.exists('data/countries.csv'):
    df_countries = pd.read_csv('data/countries.csv')
    print('The file already exists')
else:
    df_countries['name'] = df_energy_prod['country'].unique()
    df_countries['iso_code'] = df_energy_prod['iso_code'].unique()
    df_countries.sort_values(by=['name'], inplace=True)
    df_countries.reset_index(drop=True, inplace=True)
    df_countries.to_csv('data/countries.csv', index=True)

The file already exists


## Enrich Energy Data with Ember FastAPI

In [10]:
if os.path.exists('data/ember_ren_prod_23_24.json'):
    print('The file already exists')
else:
    data = ember_fetch_data(EMBER_ENDPOINTS['generation_per_year'], countries, my_api_key, base_url)
    func.save_json(data, 'data/ember_ren_prod_23_24.json')

The file already exists


In [11]:
if os.path.exists('data/ember_enrich_ren_prod_22_23_24.csv'):
    df_ember = pd.read_csv('data/ember_enrich_ren_prod_22_23_24.csv')
    print('The file already exists')
else:
    df_json_data = func.load_json('data/ember_ren_prod_23_24.json')
    df_json_filtred = func.ember_filter_data(df_json_data)
    df_json_agg = func.ember_aggregate_bioenergy_other_renewables(df_json_filtred)
    df_json_reshaped = func.ember_reshape_data(df_json_agg)
    df_json_reshaped.to_csv('data/ember_enrich_ren_prod_22_23_24.csv', index=False)

The file already exists


## Enrich GDP Data with World Bank API

In [13]:
if os.path.exists('data/forvisualization/wb_enrich_ren_prod_1965_23.csv'):
    df_wb = pd.read_csv('data/forvisualization/wb_enrich_ren_prod_1965_23.csv')
    print('The file already exists')
else:
    df_wb = func.wb_get_data(GDP_INDICATORS, countries)
    df_wb.fillna(0, inplace=True)
    # map countries index on iso_code and remove redundant columns
    df_wb['fk_country'] = df_wb['iso_code'].map(lambda iso: df_countries[df_countries['iso_code'] == iso].index[0])
    df_wb.drop(['iso_code'], axis=1, inplace=True)
    df_wb.to_csv('data/forvisualization/wb_enrich_ren_prod_1965_23.csv', index=False)

The file already exists


## Combine Datasets

In [15]:
# Combine base dataset with enriched data
if os.path.exists('data/forvisualization/enriched_energy_prod.csv'):
    df_energy = pd.read_csv('data/forvisualization/enriched_energy_prod.csv')
    print('The file already exists')
else:
    df_energy = pd.concat([df_energy_prod, df_ember])
    df_energy = df_energy.sort_values(by=['iso_code', 'year'])
    df_energy.reset_index(drop=True, inplace=True)
    df_energy.fillna(0, inplace=True)
    # map countries index on iso_code and remove redundant columns
    df_energy['fk_country'] = df_energy['iso_code'].map(lambda iso: df_countries[df_countries['iso_code'] == iso].index[0])
    df_energy.drop(['country', 'iso_code'], axis=1, inplace=True)
    df_energy.to_csv('data/forvisualization/enriched_energy_prod.csv')

The file already exists


In [119]:
if os.path.exists('data/forvisualization/combined_energy_wb.csv'):
    df_combined = pd.read_csv('data/forvisualization/combined_energy_wb.csv')
    print('The file already exists')
else:
    exclude_cols = {'year', 'fk_country'}
    columns_to_check = [col for col in df_combined2.columns if col not in exclude_cols]
    if 'Unnamed: 0' in df_energy.columns:
        df_energy.drop('Unnamed: 0', axis=1, inplace=True)
    df_combined = df_energy.merge(df_wb, on=['fk_country', 'year'], how='outer')
    df_combined.fillna(0, inplace=True)
    df_combined = df_combined.loc[(df_combined[columns_to_check] != 0).any(axis=1)]
    df_combined.to_csv('data/forvisualization/combined_energy_wb.csv')

The file already exists
